In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
pd.set_option('display.max_colwidth',100)
import nltk

In [4]:
t_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/english_hope_train.csv',names=["text","label","NaN"], sep='\t')
t_data.pop('NaN')
t_data.head(9)

,text,label
0,these tiktoks radiate gay chaotic energy and i love it,Non_hope_speech
1,@Champions Again He got killed for using false money,Non_hope_speech
2,It's not that all lives don't matter,Non_hope_speech
3,Is it really that difficult to understand? Black lives matter and all lives matter are not mutua...,Non_hope_speech
4,Whenever we say black isn't that racists? Why don't just say Americans.,Non_hope_speech
5,Ros The Boss u don’t know that she’s actually lgbtq tho so....,Non_hope_speech
6,That was funny at the end when Larry said 'What are we arguing about then'. haha but that said,Non_hope_speech
7,She saves lives with her music.,Non_hope_speech
8,There were a lot of Samoans in my Army unit,Non_hope_speech


In [5]:
#t_data['text']=t_data['text'].str.lower()       #changing into lower case       (remove and check acc too)

In [6]:
t_data['text']=t_data['text'].str.strip()       #remove white spaces
t_data['text']=t_data['text'].str.replace(r'\d+','')   #remove numbers
t_data['text']=t_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
t_data['text']=t_data['text'].str.replace('[^\w\s]','')      #removing punct

In [7]:
#removing url if any
import re
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
t_data['text']=t_data['text'].apply(lambda x:remove_URL(x))

In [8]:
#removing stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
def remove_sw(txt):
    no_sw =" ".join([a for a in txt.split() if a not in stop])
    return no_sw
t_data['text'] = t_data['text'].apply(lambda x: remove_sw(x))
t_data.head(9)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,text,label
0,tiktoks radiate gay chaotic energy love,Non_hope_speech
1,Champions Again He got killed using false money,Non_hope_speech
2,Its lives dont matter,Non_hope_speech
3,Is really difficult understand Black lives matter lives matter mutually exclusive They true gene...,Non_hope_speech
4,Whenever say black isnt racists Why dont say Americans,Non_hope_speech
5,Ros The Boss u dont know shes actually lgbtq tho,Non_hope_speech
6,That funny end Larry said What arguing haha said,Non_hope_speech
7,She saves lives music,Non_hope_speech
8,There lot Samoans Army unit,Non_hope_speech


In [9]:
#correct wrong words
#from textblob import TextBlob
#t_data['text'] = t_data['text'].apply(lambda txt: ''.join(TextBlob(txt).correct()))    #(Taking so much time)

In [10]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(t_data['label'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(t_data['label'])
label=to_categorical(np.asarray(train_labelEncode))
label[9]

['Hope_speech' 'Non_hope_speech' 'not-English']


array([1., 0., 0.], dtype=float32)

In [11]:
#counting labels      
 
t_data['label'].value_counts()
#must use something to balance data

Non_hope_speech    20778
Hope_speech         1962
not-English           22
Name: label, dtype: int64

In [12]:
#words in each line
 
t_data['totalwords'] = t_data['text'].str.count(' ') + 1
t_data

,text,label,totalwords
0,tiktoks radiate gay chaotic energy love,Non_hope_speech,6
1,Champions Again He got killed using false money,Non_hope_speech,8
2,Its lives dont matter,Non_hope_speech,4
3,Is really difficult understand Black lives matter lives matter mutually exclusive They true gene...,Non_hope_speech,47
4,Whenever say black isnt racists Why dont say Americans,Non_hope_speech,9
...,...,...,...
22757,Its load bollocks every life matters simple,Non_hope_speech,7
22758,say lives matter deku wouldnt say way bro,Non_hope_speech,8
22759,God says life matters,Non_hope_speech,4
22760,This video shit A bunch whiny ass bitches bitching nothing,Non_hope_speech,10


In [13]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [14]:
tok2=Tokenizer(char_level=True,filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n',lower=True)

In [15]:
tok1.fit_on_texts(t_data['text'])
words=len(tok1.word_counts)
words

20497

In [16]:
tok2.fit_on_texts(t_data['text'])
ltr=len(tok2.word_counts)

In [17]:
wordlist=tok1.word_index
#wordlist

In [18]:
encode=tok1.texts_to_sequences(t_data['text'])
#print(t_data['text'][5555])
#encode[5555]

In [19]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=10
padd = sequence.pad_sequences(encode, maxlen=max_length, padding='post')
padd

array([[6684, 9679,  937, ...,    0,    0,    0],
       [9680, 1467,  101, ...,  270,    0,    0],
       [  18,    2,    6, ...,    0,    0,    0],
       ...,
       [  25,  161,   33, ...,    0,    0,    0],
       [  26,   77,  409, ..., 2612, 2054,  116],
       [2654, 4317, 1147, ...,    0,    0,    0]], dtype=int32)

In [20]:
cencode=tok2.texts_to_sequences(t_data['text'])
cpadd=sequence.pad_sequences(cencode,maxlen=70,padding='post')

In [21]:
# importing libraries for creating model
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D,Input
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM,Bidirectional,GlobalAveragePooling1D
from keras.layers.merge import concatenate

#GloVe Embedding

In [22]:
embeddings_index = {}
f = open('drive/MyDrive/Summer Internship 2020 July/My Test File/Sunil/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [23]:
embedding_matrix = np.zeros((words+1, 300))
for word, i in tok1.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

#Word2 vec embeddding

In [25]:
embeddings_index1 = {}
g = open('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/DEEP L/english w2v.txt')
#g = open('drive/MyDrive/Summer Internship 2020 July/My Test File/Sunil/glove.6B.300d.txt')
for line in g:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index1[word] = coefs
g.close()
print('Loaded %s word vectors.' % len(embeddings_index1))

Loaded 23561 word vectors.


In [26]:
embedding_matrix1 = np.zeros((words+1, 300))
for word, i in tok1.word_index.items():
	embedding_vector1 = embeddings_index1.get(word)
	if embedding_vector1 is not None:
		embedding_matrix1[i] = embedding_vector1

#Embedding layer

In [27]:
Embedding_Layer = Embedding(words+1,300,weights=[embedding_matrix], input_length=max_length, trainable=False)
Embedding_Layer2=Embedding(words+1,300,weights=[embedding_matrix1], input_length=max_length, trainable=False)
Embedding_Layer3=Embedding(words+1,300,input_length=max_length)
Embedding_Layer4=Embedding(ltr+1,300,input_length=70)

#Parallel models


In [28]:
# channel 1
inputs1 = Input(shape=(max_length,))
embedding1 = Embedding_Layer(inputs1)
conv1 = LSTM(128)(embedding1)
drop1 = Dropout(0.5)(conv1)
#pool1 = MaxPooling1D(pool_size=2)(drop1)
#conv12=Conv1D(filters=64,kernel_size=4,activation='relu')(pool1)
#drop12=Dropout(0.5)(conv12)
#pool12=MaxPooling1D(pool_size=3,padding='same')(drop12)
#lstm=LSTM(128)(pool12)
flat1 =(drop1)

In [29]:
# channel 2
inputs2 = Input(shape=(max_length,))
embedding2 = Embedding_Layer2(inputs2)
conv2 = Bidirectional(LSTM(128))(embedding2)
drop2 = Dropout(0.5)(conv2)
#pool2 = MaxPooling1D(pool_size=2,padding='same')(drop2)
#conv22=Conv1D(filters=32,kernel_size=4,activation='relu')(pool2)
#drop22=Dropout(0.5)(conv22)
#pool22=MaxPooling1D(pool_size=3,padding='same')(drop22)
#lstm2=LSTM(128)(pool22)
flat2 = (drop2)

In [30]:
#Channel 3
inputs3=Input(shape=(max_length,))
embedding3=Embedding_Layer3(inputs3)
conv3=LSTM(128)(embedding3)
drop3=Dropout(0.5)(conv3)
flat3=(drop3)

In [31]:
#Channel 4
inputs4=Input(shape=(70,))
embedding4=Embedding_Layer4(inputs4)
conv4=LSTM(256)(embedding4)
drop4=Dropout(0.5)(conv4)
flat4=(drop4)

In [32]:
merged = concatenate([flat4, flat1,flat2,flat3])
dense1 = Dense(128, activation='relu')(merged)
outputs = Dense(3, activation='sigmoid')(dense1)
model = Model(inputs=[inputs4, inputs1,inputs2,inputs3], outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 70)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 10)]         0                                            
______________________________________________________________________________________________

#compile

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


#Training models

In [ ]:
model.fit([cpadd,padd,padd,padd],label,epochs=30,verbose=1,batch_size=64)

Epoch 1/30
356/356 [==============================] - 254s 691ms/step - loss: 0.2025 - acc: 0.8931
Epoch 2/30
356/356 [==============================] - 245s 689ms/step - loss: 0.1038 - acc: 0.9406
Epoch 3/30
356/356 [==============================] - 247s 693ms/step - loss: 0.0668 - acc: 0.9624
Epoch 4/30
257/356 [====================>.........] - ETA: 1:07 - loss: 0.0382 - acc: 0.9803

#Development data

In [ ]:
d_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/english_hope_dev.csv',names=['text','label','nan'],sep='\t')

In [ ]:
d_data=d_data[["text","label"]]
d_data

,text,label
0,Thats it.... like....I dont like that statue,Non_hope_speech
1,@Generation X Counting money that she been given for promoting hysterical green agenda.,Non_hope_speech
2,@Paola Hernandez i never said to be intolerant and hateful........ -_-,Non_hope_speech
3,@Firstlast300 Wow An opinion is that I don't like a color,Non_hope_speech
4,WOW!!!!!!!That was so so inspiring and incredible!!! THis speech has just saved my day,Hope_speech
...,...,...
2838,Such fake sentiment. .,Non_hope_speech
2839,@A G black lives arent undervalued compared to white lives though,Non_hope_speech
2840,People who pulled it down can and will be arrested for causing damage to public property lol.,Non_hope_speech
2841,@Aaron Castellanos It will be a two hour movie showing people watching television,Non_hope_speech


In [ ]:
#processing as training set

d_data['text']=d_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
d_data['text']=d_data['text'].str.strip()       #remove white spaces
d_data['text']=d_data['text'].str.replace(r'\d+','')   #remove numbers
d_data['text']=d_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
d_data['text']=d_data['text'].str.replace('[^\w\s]','')      #removing punct
d_data['text']=d_data['text'].apply(lambda x:remove_URL(x))
d_data['text'] = d_data['text'].apply(lambda x: remove_sw(x))


In [ ]:
encoded1 =tok1.texts_to_sequences(d_data['text'])
print(d_data['text'][0])
encoded1[0]

thats likei dont like statue


[22, 6, 7, 91]

In [ ]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([22,  6,  7, 91,  0,  0,  0,  0,  0,  0], dtype=int32)

In [ ]:
cencoded1=tok2.texts_to_sequences(d_data['text'])
cpadded=sequence.pad_sequences(cencoded1,maxlen=70,padding='post')

In [ ]:
val_labelEncode=labelEncode.transform(d_data['label'])
val_label=to_categorical(np.asarray(val_labelEncode))
val_label[0]

array([0., 1., 0.], dtype=float32)

#prediction

In [ ]:
# devlopment prediction of CNN

dev_predictions = model.predict([cpadded,padded,padded,padded])
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
dev_predictions1 = np.zeros_like(dev_predictions)
dev_predictions1[np.arange(len(dev_predictions)), dev_predictions.argmax(1)] = 1
#print(dev_predictions)
#print(dev_predictions1)

ValueError: ignored

#Acuracy report

In [ ]:
#Accuracy CNN
from sklearn.metrics import classification_report
print(classification_report(val_label,dev_predictions1))

#TESTING

In [ ]:
tx_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/english_hope_test.csv',names=['text'],sep=',')
tx_data


In [ ]:
#PREprocessing
#processing as training set

tx_data['text']=tx_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
tx_data['text']=tx_data['text'].str.strip()       #remove white spaces
tx_data['text']=tx_data['text'].str.replace(r'\d+','')   #remove numbers
tx_data['text']=tx_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
tx_data['text']=tx_data['text'].str.replace('[^\w\s]','')      #removing punct
tx_data['text']=tx_data['text'].apply(lambda x:remove_URL(x))
tx_data['text'] = tx_data['text'].apply(lambda x: remove_sw(x))


In [ ]:
encoded3 =tok1.texts_to_sequences(tx_data['text'])
padded3 = sequence.pad_sequences(encoded3, maxlen=max_length, padding='post')

In [ ]:
tencoded1=tok2.texts_to_sequences(tx_data['text'])
tpadded=sequence.pad_sequences(tencoded1,maxlen=70,padding='post')

In [ ]:
#prediction
#prediction of CNN

predictions = model.predict([tpadded,padded3,padded3,padded3])
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
predictions1 = np.zeros_like(predictions)
predictions1[np.arange(len(predictions)), predictions.argmax(1)] = 1
#print(dev_predictions)
#predictions1 = np.where(predictions>0.3,1,0)
print(predictions1)

In [ ]:
type(predictions1)
test_predictions12=np.argmax(predictions1,axis=1)

In [ ]:
test_predictions12

In [ ]:
tx_data['label']=labelEncode.inverse_transform(test_predictions12)
tx_data

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/English/english_hope_test.csv',names=['text'],sep=',')
test

In [ ]:
test['label']=labelEncode.inverse_transform(test_predictions12)
test['label'].value_counts()

#into CSV

In [ ]:
test.to_csv('1_iiit_dwd_ENGLISH.csv',index=False)